In [1]:
import pandas as pd

In [2]:
# Load datasets
index_data = pd.read_csv('Index-attention.csv')
disease_data = pd.read_csv('disease-attention.csv')

In [3]:
# Merge datasets based on the 'Date' column
combined_data = pd.merge(index_data, disease_data, on='Date', how='outer')

In [7]:
# Create undirected edges for Temporary-Temporary connections based on 'Term' values
temporary_temporary_edges = set()
for i in range(len(combined_data)):
    for j in range(i + 1, len(combined_data)):  # Ensure i < j to avoid duplicating undirected edges
        term_i = combined_data.loc[i, 'Term']
        term_j = combined_data.loc[j, 'Term']
        if term_i == term_j:
            date_i = combined_data.loc[i, 'Date']
            date_j = combined_data.loc[j, 'Date']
            temporary_temporary_edges.add((date_i, date_j, 'Temporary-Temporary', term_i))

In [9]:
# Create edges and weights for Temporary-Index connections
temporary_index_edges = []
for i in range(len(combined_data)):
    date = combined_data.loc[i, 'Date']
    
    nasdaq_change = combined_data.loc[i, 'NASDAQ']
    sp500_change = combined_data.loc[i, 'S&P 500']
    dowjones_change = combined_data.loc[i, 'Dow Jones']
    
    temporary_index_edges.append((date, 'NASDAQ', 'Temporary-Index', nasdaq_change))
    temporary_index_edges.append((date, 'S&P 500', 'Temporary-Index', sp500_change))
    temporary_index_edges.append((date, 'Dow Jones', 'Temporary-Index', dowjones_change))


In [10]:
# Convert edges to DataFrame
temporary_temporary_df = pd.DataFrame(temporary_temporary_edges, columns=['Source node', 'Target node', 'Edge Type', 'Attribute'])
temporary_index_df = pd.DataFrame(temporary_index_edges, columns=['Source node', 'Target node', 'Edge Type', 'Attribute'])

# Concatenate edge DataFrames
edges_df = pd.concat([temporary_temporary_df, temporary_index_df])

In [11]:
# Save combined dataset
edges_df.to_csv('Attention.csv', index=False)